# Juan Carlos Perez Ramirez
## Procesamiento de Lenguaje Natural
## Examen 2

### Librerias

In [6]:
import os
import xml.etree.ElementTree as ET
import re
import pandas as pd
from itertools import combinations
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/juancho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Preprocesamiento de datos

In [ ]:
# Rutas de entrada y salida
split = "test"
dir = "es_" + split
path = "../../corpus/examen2/"
truth = path + dir + '/truth_order.txt'  # archivo con los identificadores
xml_dir = path + dir  # carpeta donde están los archivos XML
docs_file = path + split + '.csv'

# abrimos los archivos de salida
with open(docs_file, 'w', encoding='utf-8') as f_textos, \
     open(truth, 'r', encoding='utf-8') as f_in:

    for linea in f_in:
        partes = linea.strip().split(":::")
        if len(partes) != 3:
            continue  # Saltamos líneas mal formateadas

        identificador = partes[0]
        genero = partes[1]
        nacionalidad = partes[2]
        archivo_xml = os.path.join(xml_dir, f"{identificador}.xml")

        if not os.path.exists(archivo_xml):
            print(f"Archivo {archivo_xml} no encontrado, saltando.")
            continue

        try:
            tree = ET.parse(archivo_xml)
            root = tree.getroot()
            documentos = root.find('documents')

            # Guardamos todos los tweets en una lista
            tweets = []
            for doc in documentos.findall('document'):
                texto = doc.text.strip()
                texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)  # elimina URLs
                texto = re.sub(r"@\w+", "<user>", texto) # sustituye menciones por "<user>"
                texto = texto.replace('\n', ' ')  # elimina saltos de línea dentro del tweet
                texto = texto.lower()  # pasar a minúsculas
                tweets.append(texto)

            if tweets:
                linea_usuario = " </s> ".join(tweets) + " </s> " + genero + " </s> " + nacionalidad + "\n"
                f_textos.write(linea_usuario)

        except Exception as e:
            print(f"Error procesando {archivo_xml}: {e}")

In [12]:
stop_words = list(set(stopwords.words('spanish')))
tokenizer = TweetTokenizer()

def k_skip_n_grams(tokens, n=2, k=1):
    skip_ngrams = []
    length = len(tokens)
    for i in range(length):
        window = tokens[i:i+k+n]
        if len(window) >= n:
            for indices in combinations(range(len(window)), n):
                if max(indices) - min(indices) <= k + n - 1:
                    ngram = [window[idx] for idx in indices]
                    skip_ngrams.append("_".join(ngram))
    return skip_ngrams

def multi_skipgrams(texto, n_vals=[2], k_vals=[1], tokenizer=tokenizer):
    tokens = tokenizer.tokenize(texto)
    all_skips = []

    for n in n_vals:
        for k in k_vals:
            skips = k_skip_n_grams(tokens, n=n, k=k)
            all_skips.extend(skips)

    return " ".join(all_skips)

def load_data(path_txt, n_skip, k):
    texts = []
    texts_skip = []
    genders = []
    nationalities = []

    with open(path_txt, 'r', encoding='utf-8') as f:
        for line in f:
            if "</s>" in line:
                partes = line.strip().split("</s>")
                if len(partes) < 3:
                    continue
                *tweets, genero, nacionalidad = partes
                texto_completo = " [SEP] ".join(tweets)
                texts.append(texto_completo)
                texts_skip.append(multi_skipgrams(texto_completo, n_vals=n_skip, k_vals=k))
                genders.append(genero)
                nationalities.append(nacionalidad)

    df = pd.DataFrame({
        'text': texts,
        'text_skip': texts_skip,
        'gender': genders,
        'nationality': nationalities
    })

    gender_map = {' male ': 0, ' female ': 1}
    nat_map = {l: i for i, l in enumerate(sorted(df['nationality'].unique()))}
    df['gender'] = df['gender'].map(gender_map)
    df['nationality'] = df['nationality'].map(nat_map)

    print(f"Nationalities: {nat_map}")
    return df, gender_map, nat_map


In [32]:
import gc

path_train = "../../corpus/examen2/train.csv"
path_val = "../../corpus/examen2/val.csv"
path_test = "../../corpus/examen2/test.csv"

df, gender_map, nat_map = load_data(path_train, n_skip=[3], k=[1])

#df_test, _, _ = load_data(path_test)

vectorizer_ngram = TfidfVectorizer(max_features=40000, ngram_range=(1, 3)) # para n-gramas
vectorizer_skip = TfidfVectorizer(max_features=10000) # para skip-grams

Nationalities: {' argentina': 0, ' chile': 1, ' colombia': 2, ' mexico': 3, ' peru': 4, ' spain': 5, ' venezuela': 6}


In [ ]:
# n-gramas
X_ngram = vectorizer_ngram.fit_transform(df['text'])

# skip-gramas
X_skip = vectorizer_skip.fit_transform(df['text_skip'])

X = hstack([X_ngram, X_skip])

y_gender = df['gender']
y_nat = df['nationality']

del X_ngram, X_skip, df

In [34]:
scaler = StandardScaler(with_mean=False)  # TF-IDF es sparse

df_val, _, _ = load_data(path_val, n_skip=[3], k=[1])
X_ngram = vectorizer_ngram.fit_transform(df_val['text']) # n-gramas
X_skip = vectorizer_skip.fit_transform(df_val['text_skip']) # skip-gramas
X_val = hstack([X_ngram, X_skip])

y_gender_val = df_val['gender']
y_nat_val = df_val['nationality']

del X_ngram, X_skip, df_val

Nationalities: {' argentina': 0, ' chile': 1, ' colombia': 2, ' mexico': 3, ' peru': 4, ' spain': 5, ' venezuela': 6}


In [37]:
scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X)
X_val_scaled = scaler.transform(X_val)

In [38]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

clf = MLPClassifier(hidden_layer_sizes=(200, 50), max_iter=300, random_state=42)
clf.fit(X, y_gender)

y_pred = clf.predict(X_val)
print("Clasificación de género:\n", classification_report(y_gender_val, y_pred))

Clasificación de género:
               precision    recall  f1-score   support

           0       0.52      0.43      0.47       420
           1       0.51      0.60      0.55       420

    accuracy                           0.52       840
   macro avg       0.52      0.52      0.51       840
weighted avg       0.52      0.52      0.51       840



In [39]:
clf = MLPClassifier(hidden_layer_sizes=(200, 50), max_iter=300, random_state=42)

clf.fit(X, y_nat)

y_pred = clf.predict(X_val)
print("Clasificación de nacionalidad:\n", classification_report(y_nat_val, y_pred))

Clasificación de nacionalidad:
               precision    recall  f1-score   support

           0       0.18      0.02      0.03       120
           1       0.40      0.02      0.03       120
           2       0.40      0.02      0.03       120
           3       0.15      0.82      0.25       120
           4       0.26      0.05      0.08       120
           5       0.17      0.03      0.06       120
           6       0.26      0.22      0.24       120

    accuracy                           0.17       840
   macro avg       0.26      0.17      0.10       840
weighted avg       0.26      0.17      0.10       840

